In [2]:
import requests
from bs4 import BeautifulSoup
import json
from tqdm import tqdm

# Data fetch 

In [3]:
'''
Get the links to the objects
'''
links = []

for page_n in tqdm(range(50)):
    http = requests.get("https://www.hemnet.se/salda/bostader?page={}".format(page_n))
    soup = BeautifulSoup(http.content)

    for a in soup.find_all('a', href=True):
        res = a['href']
        if "https" in res and "salda" in res:
            links.append(res)

len(links)

100%|██████████| 50/50 [00:39<00:00,  1.25it/s]


In [5]:
'''
Save the result to a text file
'''
with open("data/links.txt", "w") as fn:
    fn.writelines(links)

In [383]:
for link in tqdm(links):
    data = requests.get(link)
    soup = BeautifulSoup(data.text)
    mydivs = soup.findAll("div", {"class": "sold-property__map js-listing-map-sold"})
    try:
        jdata = json.loads(mydivs[0]["data-initial-data"])
        with open("data/objects/{}.txt".format(jdata['listing']['id']), "w") as fn:
            fn.writelines(data.text)
    except Exception as e:
        print(e)

 10%|█         | 261/2500 [02:11<18:11,  2.05it/s]

list index out of range


 12%|█▏        | 290/2500 [02:35<50:06,  1.36s/it]

list index out of range


 18%|█▊        | 443/2500 [03:57<12:49,  2.67it/s]

list index out of range


 37%|███▋      | 919/2500 [08:12<16:09,  1.63it/s]

list index out of range


100%|██████████| 2500/2500 [22:14<00:00,  1.87it/s]


# Data processing 

In [384]:
'''
Parse files into Json
'''

from os import listdir
from os.path import isfile, join

objects = []

for f in tqdm(listdir("data/objects")):
    with open("data/objects/{}".format(f), "r") as f:
        data = f.read()
    soup = BeautifulSoup(data)
    mydivs = soup.findAll("div", {"class": "sold-property__map js-listing-map-sold"})
    jdata = json.loads(mydivs[0]["data-initial-data"])
    objects.append(jdata)

objects[2]

100%|██████████| 2446/2446 [01:41<00:00, 24.07it/s]


{'status': 'sold',
 'listing': {'id': 1204486,
  'coordinate': [63.1817274331694, 14.639372293351753],
  'address': 'Rådhusgatan 28 A',
  'typeSummary': 'Bostadsrättslägenhet',
  'iconName': 'bostadsratt',
  'price': "<span class='sold-for'>Slutpris 1\xa0900\xa0000 kr</span>",
  'price_per_area': '20\xa0541 kr/m²',
  'fee': '4\xa0948 kr/mån',
  'sale_date': 'Såld 2020-06-15',
  'asked_price': 'Begärt pris 1\xa0950\xa0000 kr',
  'rooms': '3 rum',
  'living_space': '92,5 m²',
  'supplemental_area': None,
  'url': '/salda/lagenhet-3rum-centralt-ostersunds-kommun-radhusgatan-28-a-1204486',
  'land_area': None},
 'assets_prefix': '',
 'map_url': 'https://maps.googleapis.com/maps/api/js?libraries=geometry,places&v=3.34&client=gme-hemnetservicehns&region=SE&language=sv'}

In [385]:
import pandas as pd

df = pd.DataFrame.from_dict([t['listing'] for t in objects])
df.head()

,id,coordinate,address,typeSummary,iconName,price,price_per_area,fee,sale_date,asked_price,rooms,living_space,supplemental_area,url,land_area
0,1202014,"[59.6238363159342, 16.570181550355525]",Haga parkgata 9B,Bostadsrättslägenhet,bostadsratt,<span class='sold-for'>Slutpris 1 000 000 kr</...,20 000 kr/m²,3 833 kr/mån,Såld 2020-06-11,Begärt pris 995 000 kr,2 rum,50 m²,None,/salda/lagenhet-2rum-haga-vasteras-kommun-haga...,None
1,1203316,"[59.3191730260472, 18.03829415619227]",Heleneborgsgatan 5C,Bostadsrättslägenhet,bostadsratt,<span class='sold-for'>Slutpris 6 350 000 kr</...,96 212 kr/m²,3 944 kr/mån,Såld 2020-06-12,Begärt pris 5 790 000 kr,3 rum,66 m²,None,/salda/lagenhet-3rum-sodermalm-hogalid-stockho...,None
2,1204486,"[63.1817274331694, 14.639372293351753]",Rådhusgatan 28 A,Bostadsrättslägenhet,bostadsratt,<span class='sold-for'>Slutpris 1 900 000 kr</...,20 541 kr/m²,4 948 kr/mån,Såld 2020-06-15,Begärt pris 1 950 000 kr,3 rum,"92,5 m²",None,/salda/lagenhet-3rum-centralt-ostersunds-kommu...,None
3,1201379,"[59.43984696283136, 18.07367567305071]","Kometvägen 35, vån 3",Bostadsrättslägenhet,bostadsratt,<span class='sold-for'>Slutpris 2 800 000 kr</...,34 568 kr/m²,4 663 kr/mån,Såld 2020-06-10,Begärt pris 2 995 000 kr,3 rum,81 m²,None,/salda/lagenhet-3rum-grindtorp-taby-kommun-kom...,None
4,1201494,"[59.29511755942469, 18.10640325344144]",Ulricehamnsvägen 4,Bostadsrättslägenhet,bostadsratt,<span class='sold-for'>Slutpris 2 940 000 kr</...,56 538 kr/m²,2 904 kr/mån,Såld 2020-06-10,Begärt pris 2 495 000 kr,2 rum,52 m²,None,/salda/lagenhet-2rum-stockholms-kommun-ulriceh...,None


In [386]:
def parse_num(x, n_remove_suff=0):
    if x == None: 
        return None
    t = "".join([v for v in x if v.isnumeric() or v==","])
    t = t.replace(",", ".")
    if n_remove_suff:
        t = t[:-n_remove_suff]
    return float(t)

In [387]:
df_parsed = pd.DataFrame()

In [388]:
df_parsed["price_per_area"] = df.price_per_area.apply(lambda x: parse_num(x, 1))

In [389]:
df_parsed["rooms"] = df.rooms.apply(parse_num)

In [390]:
df_parsed["fee"] = df.fee.apply(parse_num)

In [391]:
df_parsed["living_space"] = df.living_space.apply(lambda x: parse_num(x, 1))

In [392]:
df_parsed["supplemental_area"] = df.supplemental_area.apply(lambda x: parse_num(x, 1))

In [393]:
df_parsed["price"] = df.price.apply(parse_num)

In [394]:
df_parsed["asked_price"] = df.asked_price.apply(parse_num)

In [395]:
df_parsed["land_area"] = df.land_area.apply(lambda x: parse_num(x, 1))

In [396]:
df_parsed["longitude"] = df.coordinate.apply(lambda x: x[0])
df_parsed["latitude"] = df.coordinate.apply(lambda x: x[1])

In [409]:
df_parsed["typeSummary"] = df.typeSummary

In [410]:
'''
Date related
'''

df_parsed["year"] = df.sale_date.apply(lambda x: x[5:].split("-")[0])
df_parsed["month"] = df.sale_date.apply(lambda x: x[5:].split("-")[1])
df_parsed["day"] = df.sale_date.apply(lambda x: x[5:].split("-")[2])

In [411]:
def contains_num(s):
    return any(filter(lambda x: x.isnumeric(), s))

In [412]:
def floor(x):
    try:
        x = x.lower()
        if "vån " in x:
            last = x.split("vån")[-1]
            return parse_num(last)
        if x[-2:] == "tr":
            return parse_num(x[-4:-2])
    except:
        return 0
    return 0

df_parsed["floor"] = df.address.apply(floor)

In [414]:
df_parsed.describe()

,price_per_area,rooms,fee,living_space,supplemental_area,price,asked_price,land_area,longitude,latitude,floor
count,2390.000000,2375.000000,1433.000000,2392.000000,710.000000,2.446000e+03,2.430000e+03,1.000000e+03,2446.000000,2446.000000,2446.000000
mean,36985.372385,3.403368,3728.357990,87.042559,41.252113,2.864402e+06,2.703596e+06,4.757620e+03,58.889279,15.908270,0.386263
std,25616.539904,1.720102,1322.843515,47.306871,39.878895,2.184230e+06,2.091508e+06,4.038677e+04,1.868170,2.524556,1.845948
min,982.000000,1.000000,0.000000,0.000000,0.000000,5.500000e+04,7.500000e+04,1.030000e+02,55.346335,11.176771,0.000000
25%,19672.250000,2.000000,2795.000000,55.000000,9.000000,1.495000e+06,1.375000e+06,6.515000e+02,57.731481,13.224915,0.000000
50%,30909.000000,3.000000,3655.000000,77.750000,30.000000,2.420000e+06,2.250000e+06,1.092500e+03,59.280953,16.582154,0.000000
75%,47673.750000,4.000000,4580.000000,110.000000,65.000000,3.700000e+06,3.495000e+06,1.836250e+03,59.435408,18.004513,0.000000
max,418750.000000,20.000000,9321.000000,560.000000,335.000000,2.530000e+07,2.300000e+07,1.007140e+06,67.854406,24.136828,56.000000


# Save dataset 

In [416]:
df_parsed.to_csv("data/dataset.csv")

In [404]:
# from pandas_profiling import ProfileReport

# ProfileReport(df_parsed, title="Pandas Profiling Report")